# This ipynb's function is to interpolate missing quarters between 1997 to 2007. Quarters with 3 consecutive missing periods will be ignored. Quarters after 2004-06-30 will be ignored.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import multiprocessing

In [ ]:
data = pd.read_stata("S12.dta")

In [ ]:
df = data

In [ ]:
# export all fund number and all quarters between 1997-2007
fund_no = df["fundno"].unique().tolist()
quarters = pd.date_range(start='1997-03-31', end='2007-12-31', freq='Q')

# interpolate data process and store in res
def add_value(_data,fundno,quat):
    result = []
    _fund = _data[_data["fundno"]==fundno]
    for cusip in _fund["cusip"].unique().tolist():
        _res = interprolate_fund(_fund, cusip, quat)
        result.append(_res)
    result = pd.concat(result)
    res.append(result)
    return result

# interpolate data by cusip
def interprolate_fund(fund, cusip, quarter):
    final_df = fund[fund['cusip'] == cusip]
    final_df = final_df.set_index("fdate").reindex(quarter)
    final_df["cusip_nan"] = final_df["cusip"].isna().astype(int)
    final_df[final_df.index > '2004-06-31']['cusip_nan'] =1
    final_df = final_df.interpolate("pad").reset_index()
    final_df = remove_consecutive_rows(final_df)
    return final_df.dropna(how = "any", inplace = True)

# remove the value added if there are consecutive 3 rows that are added
def remove_consecutive_rows(df):
    consecutive_count = 0
    rows_to_remove = []
    
    for index, row in df.iterrows():
        if row['cusip_nan'] == 1:
            consecutive_count += 1
        else:
            consecutive_count = 0
            
        if consecutive_count >= 3:
            rows_to_remove.extend(range(index - consecutive_count + 1, index + 1))
    
    df_cleaned = df.drop(rows_to_remove)
    return df_cleaned

# parallel processing, depends on cpu
res = []
cpus = multiprocessing.cpu_count()
print(cpus)
Parallel(n_jobs = 4)(delayed(add_value)(df, id, quarters) for id in tqdm(fund_no))

res = pd.concat(res)
res.to_stata("S12_screened.dta", version = 119)